In [1]:
%run ../common/common.ipynb

Note: you may need to restart the kernel to use updated packages.


Caution: Nodes should have been already in `cluster.yaml`

In [2]:
run_command("kind delete cluster", shell=False)
run_command("kind create cluster --config=cluster.yaml", shell=False)
run_command("sleep 30", shell=False)

kind delete cluster
kind delete cluster


FileNotFoundError: [Errno 2] No such file or directory: 'kind'

In [3]:
run_command('../common/common_modules.sh')
!kubectl apply -f https://raw.githubusercontent.com/kubernetes/ingress-nginx/main/deploy/static/provider/kind/deploy.yaml

../common/common_modules.sh
../common/common_modules.sh
Log 0 - 2023-08-19 17:58:13 : Warning: rbac.authorization.k8s.io/v1beta1 ClusterRoleBinding is deprecated in v1.17+, unavailable in v1.22+; use rbac.authorization.k8s.io/v1 ClusterRoleBinding
Log 0 - 2023-08-19 17:58:13 : clusterrolebinding.rbac.authorization.k8s.io/manager-full created
Log 0 - 2023-08-19 17:58:15 : customresourcedefinition.apiextensions.k8s.io/certificaterequests.cert-manager.io created
Log 0 - 2023-08-19 17:58:15 : customresourcedefinition.apiextensions.k8s.io/certificates.cert-manager.io created
Log 0 - 2023-08-19 17:58:15 : customresourcedefinition.apiextensions.k8s.io/challenges.acme.cert-manager.io created
Log 0 - 2023-08-19 17:58:15 : customresourcedefinition.apiextensions.k8s.io/clusterissuers.cert-manager.io created
Log 0 - 2023-08-19 17:58:16 : customresourcedefinition.apiextensions.k8s.io/issuers.cert-manager.io created
Log 0 - 2023-08-19 17:58:16 : customresourcedefinition.apiextensions.k8s.io/orders.a

In [4]:
(manager_node, jobmanager_node, taskmanager_nodes) = get_label_nodes(ip_address=True)

kind-control-plane	{'beta.kubernetes.io/arch': 'amd64', 'beta.kubernetes.io/os': 'linux', 'kubernetes.io/arch': 'amd64', 'kubernetes.io/hostname': 'kind-control-plane', 'kubernetes.io/os': 'linux', 'node-role.kubernetes.io/control-plane': '', 'node-role.kubernetes.io/master': '', 'node.kubernetes.io/exclude-from-external-load-balancers': ''}
kind-worker	{'beta.kubernetes.io/arch': 'amd64', 'beta.kubernetes.io/os': 'linux', 'ingress-ready': 'true', 'kubernetes.io/arch': 'amd64', 'kubernetes.io/hostname': 'kind-worker', 'kubernetes.io/os': 'linux', 'tier': 'manager'}
kind-worker2	{'beta.kubernetes.io/arch': 'amd64', 'beta.kubernetes.io/os': 'linux', 'kubernetes.io/arch': 'amd64', 'kubernetes.io/hostname': 'kind-worker2', 'kubernetes.io/os': 'linux', 'tier': 'jobmanager'}
kind-worker3	{'beta.kubernetes.io/arch': 'amd64', 'beta.kubernetes.io/os': 'linux', 'kubernetes.io/arch': 'amd64', 'kubernetes.io/hostname': 'kind-worker3', 'kubernetes.io/os': 'linux', 'tier': 'taskmanager'}


In [5]:
!kubectl describe ing -n manager

Name:             grafana
Labels:           <none>
Namespace:        manager
Address:          
Ingress Class:    <none>
Default backend:  <default>
Rules:
  Host                        Path  Backends
  ----                        ----  --------
  grafana.127-0-0-1.sslip.io  
                              /   prom-grafana:80 ()
Annotations:                  kubernetes.io/ingress.class: nginx
                              nginx.ingress.kubernetes.io/proxy-body-size: 0
Events:                       <none>


Name:             minio
Labels:           <none>
Namespace:        manager
Address:          
Ingress Class:    <none>
Default backend:  <default>
Rules:
  Host                      Path  Backends
  ----                      ----  --------
  minio.127-0-0-1.sslip.io  
                            /   minio:9000 (10.244.3.8:9000)
Annotations:                kubernetes.io/ingress.class: nginx
                            nginx.ingress.kubernetes.io/proxy-body-size: 0
Events:              

In [6]:
address = "127-0-0-1"
print("Manager node: {}:30080".format(manager_node))
print("Access to Minio: http://minio.{}.nip.io:30080".format(address))
print("Access to Grafana: http://grafana.{}.nip.io:30080".format(address))
print("Access to Prometheus: http://prometheus.{}:30080".format(address))
print("Job manager address: {}".format(jobmanager_node))
print("Task manager addresses: {}".format(taskmanager_nodes))

Manager node: 172.18.0.2:30080
Access to Minio: http://minio.127-0-0-1.nip.io:30080
Access to Grafana: http://grafana.127-0-0-1.nip.io:30080
Access to Prometheus: http://prometheus.127-0-0-1:30080
Job manager address: 172.18.0.5
Task manager addresses: 172.18.0.3


In [7]:
print("Access to Minio: http://{}:30900".format(manager_node))
print("Access to Grafana: http://{}:30300".format(manager_node))
print("Access to Prometheus: http://{}:30090".format(manager_node))
print("Job manager address: {}".format(jobmanager_node))
print("Task manager addresses: {}".format(taskmanager_nodes))

Access to Minio: http://172.18.0.2:30900
Access to Grafana: http://172.18.0.2:30300
Access to Prometheus: http://172.18.0.2:30090
Job manager address: 172.18.0.5
Task manager addresses: 172.18.0.3


In [8]:
%%bash
pwd
export ZEPPELIN_POD=`kubectl get pods -n manager -o=name | grep zeppelin | sed "s/^.\{4\}//"` 
kubectl wait -n manager pod/$ZEPPELIN_POD --for condition=Ready --timeout=600s
export MANAGER_NODE=`kubectl get nodes --selector=tier=manager -o jsonpath='{$.items[*].status.addresses[?(@.type=="InternalIP")].address}'`

kubectl -n manager exec $ZEPPELIN_POD -- wget -P /tmp https://repo.maven.apache.org/maven2/org/apache/flink/flink-sql-connector-kafka_2.12/1.14.2/flink-sql-connector-kafka_2.12-1.14.2.jar
# nexmark and the rate limit connector should be both in the tmp directory
kubectl cp -n manager ../../tmp/flink-sql-connector-kafka-ratelimit_2.12-1.14.2.jar $ZEPPELIN_POD:/tmp/flink-sql-connector-kafka-ratelimit_2.12-1.14.2.jar
kubectl cp -n manager ../../tmp/nexmark-flink-0.2-SNAPSHOT.jar $ZEPPELIN_POD:/tmp/nexmark-flink-0.2-SNAPSHOT.jar
# set Flink interpreter to isolated mode (with default parameters)
curl -X PUT  ${MANAGER_NODE}:30088/api/interpreter/setting/flink -H 'Content-Type: application/json' -d @../common/zeppelin-flink-config.json

# upload zeppelin notebooks
for FILE in ../../experiments/zeppelin/*.zpln; do echo $FILE; 
    JOB_ID=`curl ${MANAGER_NODE}:30088/api/notebook/import -d @$FILE |jq -r .body`;
    curl -X POST ${MANAGER_NODE}:30088/api/notebook/job/${JOB_ID};
done
# remove from 
kubectl -n manager exec $ZEPPELIN_POD  -- rm /opt/flink/lib/flink-connector-kafka_2.12-1.14.2.jar
kubectl -n manager exec $ZEPPELIN_POD  -- rm /opt/flink/lib/flink-sql-connector-kafka_2.12-1.14.2.jar
curl -u "admin:prom-operator" -X POST ${MANAGER_NODE}:30300/api/dashboards/import -H 'Content-Type: application/json' -d "{\"Dashboard\":$(cat ../common/grafana.json)}"

/home/guillaume/research/gepiciad/StreamBed/infra/kind
pod/zeppelin-server-75767d75dd-pbz7w condition met
{"status":"OK","message":"","body":{"id":"flink","name":"flink","group":"flink","properties":{"FLINK_HOME":{"name":"FLINK_HOME","value":"","type":"string","description":"Location of flink distribution"},"HADOOP_CONF_DIR":{"name":"HADOOP_CONF_DIR","value":"","type":"string","description":"Location of hadoop conf (core-site.xml, hdfs-site.xml and etc.)"},"HIVE_CONF_DIR":{"name":"HIVE_CONF_DIR","value":"","type":"string","description":"Location of hive conf (hive-site.xml)"},"flink.execution.mode":{"name":"flink.execution.mode","value":"local","type":"string","description":"Execution mode, it could be local|remote|yarn"},"flink.execution.remote.host":{"name":"flink.execution.remote.host","value":"","type":"string","description":"Host name of running JobManager. Only used for remote mode"},"flink.execution.remote.port":{"name":"flink.execution.remote.port","value":"","type":"number","d

Defaulted container "zeppelin-server" out of: zeppelin-server, zeppelin-server-gateway, dnsmasq, flink-downloader (init)
--2023-08-19 16:06:58--  https://repo.maven.apache.org/maven2/org/apache/flink/flink-sql-connector-kafka_2.12/1.14.2/flink-sql-connector-kafka_2.12-1.14.2.jar
Resolving repo.maven.apache.org (repo.maven.apache.org)... 151.101.8.215, 2a04:4e42:2::215
Connecting to repo.maven.apache.org (repo.maven.apache.org)|151.101.8.215|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3703507 (3.5M) [application/java-archive]
Saving to: ‘/tmp/flink-sql-connector-kafka_2.12-1.14.2.jar’

     0K .......... .......... .......... .......... ..........  1% 2.74M 1s
    50K .......... .......... .......... .......... ..........  2% 8.76M 1s
   100K .......... .......... .......... .......... ..........  4% 4.06M 1s
   150K .......... .......... .......... .......... ..........  5% 5.71M 1s
   200K .......... .......... .......... .......... ..........  6% 16.7M 1